## Setting

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     |████████████████████████████████| 954 kB 5.1 MB/s 
     |████████████████████████████████| 138 kB 36.4 MB/s 
     |████████████████████████████████| 356 kB 60.2 MB/s 
     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 3.5 MB 31.6 MB/s 
     |████████████████████████████████| 54 kB 3.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.down

In [ ]:
!pip install pyOpenSSL

In [ ]:
import ssl
print(ssl.OPENSSL_VERSION)

OpenSSL 1.1.1  11 Sep 2018


In [ ]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
import time, os
from selenium import webdriver
import requests
from bs4 import BeautifulSoup

from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.alert import Alert
import time
from tqdm.notebook import tqdm
from pandas import Series as Series
from pandas import DataFrame as DataFrame

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## Crawler

In [ ]:
save_dir = '/content/drive/MyDrive/TR_코로나 데이터분석/data'

In [ ]:
from requests import ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError

In [ ]:
# 데이터 프레임 생성
raw_df = DataFrame(columns=['날짜', '네이버/티스토리', '관광 명소', '제목', '내용', 'url', '글쓴이'])

# 드라이버 지정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

# 페이지 수 초기화
page = 1
# limit_page = 31 # 마지막 페이지.. 수동 확인
article_cnt = 1

keyword = "비양도"

while article_cnt <= 200:
    while (article_cnt <= 200):
        url = "https://search.daum.net/search?w=blog&f=section&SA=tistory&lpp=10&nil_src=tistory&q={}&period=u&sd=20200301000000&ed=20211001235959&p={}".format(keyword, page)
        driver.get(url)
        response = requests.get(url, verify=True)
        soup = BeautifulSoup(response.text, 'html.parser')

        # t_len: 한 페이지 안의 총 기사 개수
        t_len = len(driver.find_elements_by_css_selector(".type_fulltext .list_info li"))

        # 한 페이지 안의 기사 수 만큼 탐색
        for i in tqdm(range(1, t_len+1)):
            if t_len == 1:
                a = driver.find_element_by_css_selector(".type_fulltext li .wrap_tit .f_link_b")
                b = driver.find_element_by_css_selector(".type_fulltext .wrap_cont .info")
                c = driver.find_element_by_css_selector(".type_fulltext .wrap_cont .date")

            else:
                a = driver.find_element_by_css_selector(".type_fulltext li:nth-child({}) .wrap_tit .f_link_b".format(i))
                b = driver.find_element_by_css_selector(".type_fulltext li:nth-child({}) .wrap_cont .info".format(i))
                c = driver.find_element_by_css_selector(".type_fulltext li:nth-child({}) .wrap_cont .date".format(i))

            # 제목 수집
            tit = a.text
            
            # url, writer
            txt = b.text
            url_writer = txt.split('|')[0]
            url = 'https://'+ url_writer.split()[0]
            writer = "".join(url_writer.split()[1:])

            # date
            date = c.text

            # 개별 기사 클릭
            a.send_keys(Keys.ENTER)

            # 본문
            content_list = []
            # time.sleep(2)
            try:
                response = requests.get(url)
            except (ConnectTimeout, HTTPError, ReadTimeout, Timeout, ConnectionError):
                continue

            soup = BeautifulSoup(response.text, 'html.parser')
            tags = soup.select('div.tt_article_useless_p_margin p')
            for tag in tags:
                content = tag.text
                content = content.replace('\xa0', '')
                content_list.append(content)
                
            contents = ''.join(content_list)

            # 데이터 프레임에 삽입
            # 날짜, 네이버/티스토리, 관광 명소, 제목, 내용, url, 글쓴이
            raw_df.loc[article_cnt] = [date, '티스토리', keyword, tit, contents, url, writer]
            article_cnt += 1
        page += 1
            
driver.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead


  0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
page

22

In [ ]:
print(len(raw_df))
raw_df.head()

209


,날짜,네이버/티스토리,관광 명소,제목,내용,url,글쓴이
1,2021.06.17,티스토리,비양도,제주도 비양도 백패킹,"지난 5월 21-22일. 처음으로 백패킹을 다녀왔다.오래전부터 계획하고 있었지만, ...",https://feelland16.tistory.com/79,FEELLAND
2,2021.05.29,티스토리,비양도,천년의 섬 제주-비양도 방문기!,안녕하세요!오늘은 #비양도 방문기를 공유하려고 합니다!비양도는 제주 서쪽에 위치한 ...,https://noelchoe.tistory.com/88,NOEL'BLOG
3,2021.06.12,티스토리,비양도,어렵게 찾아간 제주도 비양도 여행,2021년 5월 26일. 드디어 제주도로 떠났다. 당초 2박 3일 여행을 계획했는데...,https://forculture.tistory.com/834,풀소리의자유로운세...
4,2021.07.18,티스토리,비양도,"제주도 섬여행 비양도를 걷다~! (한림항 주차과 배편 그리고 맛집, 풍경)",제주도여행금능해수욕장에서 뒤에 보이는~ 저기 작은 섬!오늘은 저기 보이는 비양도에 ...,https://1dingding.tistory.com/336,여행일기와카페투어
5,2021.09.04,티스토리,비양도,"하는 제주도 가족여행 : 우도 - 비양도, 검멀레해수욕장, 서빈백사해수욕장",성산포 종합 여객터미널몇년 전에 우도 갔을 때는 렌트카를 주차장에 세워놓고 우도에 ...,https://tired-panda.tistory.com/93,백수판다의라이프...


In [ ]:
raw_df[raw_df['내용'].str.contains('코로나| covid| 펜데믹| 거리두기| 코로롱| 마스크| 턱스크')].shape[0]

28

In [ ]:
raw_df.to_csv(save_dir+'/{}_data.csv'.format(keyword), index=False)

In [ ]:
# 데이터 프레임 생성
raw_df = DataFrame(columns=['날짜', '네이버/티스토리', '관광 명소', '제목', '내용', 'url', '글쓴이'])

# 드라이버 지정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

# 페이지 수 초기화
page = 1
limit_page = 31 # 마지막 페이지.. 수동 확인
article_cnt = 1

keyword = "중문색달해변"

while article_cnt <= 200:
    while (article_cnt <= 200):
        url = "https://search.daum.net/search?w=blog&f=section&SA=tistory&lpp=10&nil_src=tistory&q={}&period=u&sd=20200301000000&ed=20211001235959&p={}".format(keyword, page)
        driver.get(url)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # pages = len(driver.find_elements_by_class_name("link_page"))
        pages = 57

        # t_len: 한 페이지 안의 총 기사 개수
        t_len = len(driver.find_elements_by_css_selector(".type_fulltext .list_info li"))

        # 한 페이지 안의 기사 수 만큼 탐색
        for i in tqdm(range(1, t_len+1)):
            if t_len == 1:
                a = driver.find_element_by_css_selector(".type_fulltext li .wrap_tit .f_link_b")
                b = driver.find_element_by_css_selector(".type_fulltext .wrap_cont .info")
                c = driver.find_element_by_css_selector(".type_fulltext .wrap_cont .date")

            else:
                a = driver.find_element_by_css_selector(".type_fulltext li:nth-child({}) .wrap_tit .f_link_b".format(i))
                b = driver.find_element_by_css_selector(".type_fulltext li:nth-child({}) .wrap_cont .info".format(i))
                c = driver.find_element_by_css_selector(".type_fulltext li:nth-child({}) .wrap_cont .date".format(i))

            # 제목 수집
            tit = a.text
            
            # url, writer
            txt = b.text
            url_writer = txt.split('|')[0]
            url = 'https://'+ url_writer.split()[0]
            writer = "".join(url_writer.split()[1:])

            # date
            date = c.text

            # 개별 기사 클릭
            a.send_keys(Keys.ENTER)

            # 본문
            content_list = []
            try:
                response = requests.get(url)
            except (ConnectTimeout, HTTPError, ReadTimeout, Timeout, ConnectionError):
                continue
            soup = BeautifulSoup(response.text, 'html.parser')
            tags = soup.select('div.tt_article_useless_p_margin p')
            for tag in tags:
                content = tag.text
                content = content.replace('\xa0', '')
                content_list.append(content)
                
            contents = ''.join(content_list)

            # 데이터 프레임에 삽입
            # 날짜, 네이버/티스토리, 관광 명소, 제목, 내용, url, 글쓴이
            raw_df.loc[article_cnt] = [date, '티스토리', keyword, tit, contents, url, writer]
            article_cnt += 1
            
            # print(date)
            # print(tit)
            # print("-" * 20)

        page += 1
            
driver.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead


  0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
print(len(raw_df))
raw_df.head()

201


,날짜,네이버/티스토리,관광 명소,제목,내용,url,글쓴이
1,2021.09.30,티스토리,중문색달해변,중문 색달해변 카페바나나 빙수 대박,남편과 제주도 여행중이었어요 !저번에 친구들과 여행에서 백종원호텔이라고 불리우는 더...,https://yundaisy.tistory.com/64,일상리뷰의모든것
2,2021.06.29,티스토리,중문색달해변,제주에서 즐기는 액티비티 서핑 - 중문색달해변 한옥서핑샵 '중문서핑스쿨',중문색달해변안녕하세요!제주도는 요즘 해도 길어지고정말 여름이 온 것 같아요!제주 하...,https://ye-un.tistory.com/56,Blog_ye
3,2021.05.24,티스토리,중문색달해변,200919 - 중문색달해변 노을,점점 노을이 지는 타이밍이 딱이었다구름도 거의 없어서 꽤 이쁠것같은 느낌근처에 클리...,https://barsike.tistory.com/665,진정한자유를원하는...
4,2021.06.09,티스토리,중문색달해변,"[퇴사 후 제주 여행] 10일 차 : 중문색달해변, 엉덩물계곡, 넘버피넛",,https://nemonemo.tistory.com/11,지금이아니면안되는...
5,2021.03.22,티스토리,중문색달해변,서핑의 메카는 바로 여기 ! 제주 여행지 추천 - 중문색달해변 360VR,"날씨가 다시 쌀쌀해지고문득 여름이 그리워지는 요즘,보기만 해도 활기찬 서핑은감상하는...",https://look360.tistory.com/63,www.look360.kr


In [ ]:
raw_df.to_csv(save_dir+'/{}_data.csv'.format(keyword), index=False)